In [2]:
import urllib.request
url = ("https://raw.githubusercontent.com/rasbt/LLMs-from-scratch/main/ch02/01_main-chapter-code/the-verdict.txt")
file_path = "the-verdict2.txt"
urllib.request.urlretrieve(url, file_path)

('the-verdict2.txt', <http.client.HTTPMessage at 0x7f2cfed0bf40>)

In [3]:
with open(file_path, "r", encoding="utf-8") as f:
    raw_text = f.read()

print("Total num char", len(raw_text))
print(raw_text[:99])

Total num char 20479
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no 


In [4]:
#import tiktoken
# use Byte Pair Encoding
from importlib.metadata import version
import tiktoken
print("tiktoken version:", version("tiktoken"))

tiktoken version: 0.8.0


In [5]:
tokenizer = tiktoken.get_encoding("gpt2")
text = ("laudalund")


In [6]:
integers = tokenizer.encode(text, allowed_special={"<|endoftext|>"})
print(integers)

[75, 3885, 282, 917]


In [7]:
for i in integers:
    print(tokenizer.decode([i]))

l
aud
al
und


In [8]:
#input-target pair building
raw_text
enc_text = tokenizer.encode(raw_text)
print(len(enc_text))


5145


In [9]:
enc_sample = enc_text[50:]

context_size = 4
x = enc_sample[:context_size]
y = enc_sample[1:context_size+1]
print(x)
print(y)

[290, 4920, 2241, 287]
[4920, 2241, 287, 257]


In [10]:
import torch
from torch.utils.data import Dataset, DataLoader

class GPTDatasetV1(Dataset):
    def __init__(self, txt, tokenizer, max_length, stride):
        self.input_ids = []
        self.target_ids = []
        token_ids = tokenizer.encode(txt)

        for i in range(0, len(token_ids) - max_length, stride):
            input_chunk = token_ids[i:i + max_length]
            target_chunk = token_ids[i + 1: i + max_length + 1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))
    def __len__(self):
        return len(self.input_ids)
    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]

In [11]:
max_length = 4
stride = 1

dataLoader = GPTDatasetV1(raw_text, tokenizer, max_length, stride)

In [27]:
def create_dataloader_v1(txt, batch_size=4, max_length=256,
    stride=128, shuffle=True, drop_last=True,
    num_workers=0):
    tokenizer = tiktoken.get_encoding("gpt2")
    dataset = GPTDatasetV1(txt, tokenizer, max_length, stride)
    dataloader = DataLoader(
                    dataset,
                    batch_size=batch_size,
                    shuffle=shuffle,
                    drop_last=drop_last,
                    num_workers=num_workers
                )
    return dataloader

In [32]:
dataloader = create_dataloader_v1(
    raw_text,batch_size=4,max_length=4,stride=1, shuffle=False)

data_iter = iter(dataloader)
inputs, outputs = next(data_iter)
print(inputs)

tensor([[  40,  367, 2885, 1464],
        [ 367, 2885, 1464, 1807],
        [2885, 1464, 1807, 3619],
        [1464, 1807, 3619,  402]])


In [33]:
vocab_size = 50257
output_dim = 256
token_embedding_layer = torch.nn.Embedding(vocab_size,output_dim)

In [34]:
token_embedding_layer.weight.shape

torch.Size([50257, 256])

In [38]:
max_length = 4
dataloader = create_dataloader_v1(
    raw_text, batch_size=8, max_length=max_length,
    stride=max_length,shuffle=False
)

data_iter = iter(dataloader)
inputs, outputs = next(data_iter)
print("inputs", inputs.shape)
print("outputs", outputs.shape)

inputs torch.Size([8, 4])
outputs torch.Size([8, 4])


In [39]:
toekn_embeddings = token_embedding_layer(inputs)
toekn_embeddings.shape

torch.Size([8, 4, 256])

In [40]:
context_length = max_length
pos_embedding_layer = torch.nn.Embedding(context_length, output_dim)
pos_embedding = pos_embedding_layer(torch.arange(context_length))
pos_embedding.shape

torch.Size([4, 256])

In [42]:
torch.arange(4) + 5

tensor([5, 6, 7, 8])

In [43]:
input_embeddings = toekn_embeddings + pos_embedding
input_embeddings.shape

torch.Size([8, 4, 256])